# Mini Project

- Purpose: Community Detection.
- Features:
  -Mature content
    - Marking your Twitch channel as "Mature Content":
 Swearing, Violence, Drugs and alcohol, Sexual content
- Type: Binary

- Other feature: **Twitch Partners** are creators who stream a variety of content, from games, music, talk shows, art, to just about anything else you can imagine. If you love sharing what you do with a live community, and have built up a sizable audience on Twitch or other services, we want to hear from you!

In [ ]:
!pip install karateclub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.4 MB/s eta 0:00:00
  Created wheel for karateclub: filename=karateclub-1.3.3-py3-none-any.whl size=101984 sha256=40771d3312fc9dbbbbf0b216cfd20af76ec4850dc6a2cb45f460edfac5545eb4
  Stored in directory: /root/.cache/pip/wheels/62/bd/af/17e7ca6ba0ed144d22502780f5c0660a8e4985939dc6973a81
Successfully built karateclub
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.2

In [ ]:
import pandas as pd
import numpy as np
import json
import community
import networkx as nx

from karateclub import DeepWalk, SCD, EgoNetSplitter
from karateclub import LabelPropagation
from sklearn.metrics import normalized_mutual_info_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset Treatment

In [ ]:
# Shared Folder Datasets

edges_df = pd.read_csv("musae_ENGB_edges.csv")
target_df = pd.read_csv("musae_ENGB_target.csv")

display(edges_df)
display(target_df)

,from,to
0,6194,255
1,6194,980
2,6194,2992
3,6194,2507
4,6194,986
...,...,...
35319,6922,7122
35320,6562,7089
35321,7069,7070
35322,7069,7110


,id,days,mature,views,partner,new_id
0,73045350,1459,False,9528,False,2299
1,61573865,1629,True,3615,False,153
2,171688860,411,True,46546,False,397
3,117338954,953,True,5863,False,5623
4,135804217,741,True,5594,False,5875
...,...,...,...,...,...,...
7121,23831198,2624,False,3174,False,3794
7122,41433855,2035,True,3158,False,6534
7123,75862831,1418,True,3839,False,2041
7124,41041370,2046,True,6208,False,6870


## Mature Feature

In [ ]:
feature_twich = target_df[['new_id', 'mature']]
display(feature_twich)

,new_id,mature
0,2299,False
1,153,True
2,397,True
3,5623,True
4,5875,True
...,...,...
7121,3794,False
7122,6534,True
7123,2041,True
7124,6870,True


In [ ]:
feature_twich['is_mature'] = feature_twich['mature'].map(lambda row: 1 if row == True else 0)

<ipython-input-5-f6d95062aa91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_twich['is_mature'] = feature_twich['mature'].map(lambda row: 1 if row == True else 0)


In [ ]:
feature_twich = feature_twich.rename({'new_id':'id'}, axis=1)

In [ ]:
feature_twich

,id,mature,is_mature
0,2299,False,0
1,153,True,1
2,397,True,1
3,5623,True,1
4,5875,True,1
...,...,...,...
7121,3794,False,0
7122,6534,True,1
7123,2041,True,1
7124,6870,True,1


In [ ]:
feature_twich.sort_values(by=['id'])

,id,mature,is_mature
177,0,True,1
3658,1,False,0
5858,2,True,1
1696,3,False,0
3634,4,True,1
...,...,...,...
5903,7121,False,0
2757,7122,True,1
7118,7123,False,0
4383,7124,False,0


In [ ]:
is_mature = feature_twich['is_mature']
is_mature = is_mature.to_list()
is_mature[0:10] # ... etc, etc

[0, 1, 1, 1, 1, 0, 0, 1, 1, 0]

**Demonstration that we need to use the new id**

In [ ]:
inner_join_df = pd.merge(edges_df, target_df, left_on='from', right_on='id', how='inner')
print(inner_join_df)


Empty DataFrame
Columns: [from, to, id, days, mature, views, partner, new_id]
Index: []


## Characterization of Dataset

In [ ]:
display(edges_df)# Shared Folder Datasets

edges_df = pd.read_csv("musae_ENGB_edges.csv")
target_df = pd.read_csv("musae_ENGB_target.csv")

X = nx.Graph()

for index, row in edges_df.iterrows():
    X.add_edge(row["from"], row["to"])

print("Number of nodes:", X.number_of_nodes())
print("Number of edges:", X.number_of_edges())
print("List of nodes:", list(X.nodes()))
print("List of edges:", list(X.edges()))

,from,to
0,6194,255
1,6194,980
2,6194,2992
3,6194,2507
4,6194,986
...,...,...
35319,6922,7122
35320,6562,7089
35321,7069,7070
35322,7069,7110


Number of nodes: 7126
Number of edges: 35324
List of nodes: [6194, 255, 980, 2992, 2507, 986, 4003, 0, 82, 15, 343, 4282, 5442, 1162, 3401, 7106, 6211, 6611, 1348, 2239, 2, 5842, 3, 3363, 1277, 3739, 6882, 4947, 5740, 5550, 4, 4087, 5, 2481, 5269, 1, 6, 6072, 3996, 448, 1673, 3079, 436, 2126, 743, 4115, 6286, 7, 2787, 1339, 8, 6926, 9, 3756, 1773, 5945, 5424, 474, 2872, 2439, 6121, 2352, 6168, 3258, 10, 6269, 11, 4291, 6536, 2054, 2677, 12, 5237, 2176, 13, 4198, 3322, 6553, 3418, 1615, 888, 1654, 4945, 5630, 5032, 23, 2339, 6726, 4949, 14, 615, 2779, 1089, 3846, 3815, 4302, 3706, 5470, 4486, 2811, 3826, 1536, 1044, 1960, 4639, 5453, 4024, 231, 2922, 2381, 954, 6448, 1376, 660, 2292, 3532, 1883, 5468, 6716, 1132, 4710, 7051, 3180, 3235, 6343, 5212, 887, 4406, 3241, 4319, 259, 3807, 1390, 1115, 16, 2444, 5892, 6315, 6079, 1752, 104, 581, 7069, 6136, 18, 4140, 19, 485, 365, 3731, 4967, 6682, 2819, 5146, 5358, 3325, 20, 559, 4482, 2156, 7054, 2580, 6299, 3704, 21, 745, 1414, 710, 4960, 470

### Descriptive statistics

In [ ]:
density = nx.density(X)
print("Density:",round(density,4))

transitivity = nx.transitivity(X)
print("Transitivity",round(transitivity,3))

try:
    diameter = nx.diameter(X)
    print("Diameter:", diameter)
except nx.NetworkXError:
    print("The graph is disconexus, it does'nt have diameter.")

## Community Detection

### Non-Overlapping
- **Label Prop**

In [ ]:

# Creating a graph this time using nx pandas reader
G = nx.from_pandas_edgelist(edges_df, 'from', 'to')


# Initialize and run the Label Propagation model
model = LabelPropagation()
model.fit(G)

# Get prediction
cluster_membership = model.get_memberships()


In [ ]:

for idx, key in enumerate(cluster_membership.keys()):
  if idx < 200 and idx % 15 == 0:
    cl_id = cluster_membership[key]
    print(f'Node: {key} - Cluster ID: {cl_id}')


Node: 2915 - Cluster ID: 253
Node: 3657 - Cluster ID: 369
Node: 5321 - Cluster ID: 1453
Node: 3517 - Cluster ID: 253
Node: 4519 - Cluster ID: 5258
Node: 1181 - Cluster ID: 253
Node: 1656 - Cluster ID: 253
Node: 2733 - Cluster ID: 253
Node: 6796 - Cluster ID: 253
Node: 578 - Cluster ID: 1453
Node: 6656 - Cluster ID: 581
Node: 3841 - Cluster ID: 253
Node: 1617 - Cluster ID: 5258
Node: 98 - Cluster ID: 253


In [ ]:
predicted_labels = [cluster_membership[node] for node in range(len(cluster_membership))]
print(predicted_labels)

[253, 253, 253, 253, 253, 253, 253, 47, 253, 253, 253, 1453, 73, 253, 253, 253, 253, 253, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 3487, 253, 253, 253, 253, 253, 253, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 530, 253, 253, 253, 253, 253, 253, 253, 581, 1453, 253, 253, 253, 253, 253, 253, 529, 1453, 253, 253, 6683, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 933, 253, 253, 253, 253, 253, 253, 253, 963, 964, 968, 253, 1453, 253, 253, 1009, 253, 253, 253, 253, 253, 253, 1054, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 54, 253, 253, 253, 1168, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 3487, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 253, 253, 1664, 253, 253, 253, 253, 253, 963, 253, 253, 253, 253, 1453, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 

## NMI

In [ ]:
# Calculate NMI
nmi_score = normalized_mutual_info_score(is_mature, predicted_labels)

print(f"{nmi_score:.4f}")


0.0223


## With iterations
- `num_runs = 100`


In [ ]:
(average_nmi, std_error_nmi) = iterate_nmi(graph=G, num_runs=100)

In [ ]:
print(f'average_nmi = {average_nmi}\nstd_error_nmi = {std_error_nmi}')

average_nmi = 0.02810741700375303
std_error_nmi = 3.4694469519536144e-19


### Overlapping

In [ ]:
model2 = SCD()
model2.fit(X)
scd_memberships = model2.get_memberships()

In [ ]:
print(scd_memberships)

{6194: 1515, 255: 1515, 980: 420, 2992: 3880, 2507: 454, 986: 913, 4003: 1583, 0: 3879, 82: 3879, 15: 1682, 343: 733, 4282: 278, 5442: 573, 1162: 876, 3401: 97, 7106: 3878, 6211: 886, 6611: 469, 1348: 1053, 2239: 1190, 2: 3877, 5842: 887, 3: 1681, 3363: 212, 1277: 1180, 3739: 596, 6882: 1815, 4947: 1681, 5740: 1869, 5550: 277, 4: 3876, 4087: 1171, 5: 1370, 2481: 458, 5269: 434, 1: 950, 6: 1743, 6072: 1118, 3996: 39, 448: 428, 1673: 668, 3079: 1814, 436: 3875, 2126: 1747, 743: 671, 4115: 1743, 6286: 878, 7: 3874, 2787: 2397, 1339: 3063, 8: 1386, 6926: 1386, 9: 1514, 3756: 416, 1773: 454, 5945: 454, 5424: 1514, 474: 1380, 2872: 407, 2439: 441, 6121: 407, 2352: 401, 6168: 336, 3258: 1178, 10: 3873, 6269: 3873, 11: 787, 4291: 380, 6536: 345, 2054: 209, 2677: 450, 12: 786, 5237: 786, 2176: 3872, 13: 1539, 4198: 11, 3322: 442, 6553: 1540, 3418: 234, 1615: 317, 888: 1886, 1654: 3156, 4945: 974, 5630: 47, 5032: 1539, 23: 407, 2339: 1063, 6726: 3871, 4949: 410, 14: 1565, 615: 988, 2779: 285, 10

In [ ]:
# Convert the dictionary to a format where each node has its corresponding cluster assigned
node_cluster_mapping = {node: cluster_index for node, cluster_index in scd_memberships.items()}

#possible_memberships = list(edges_df["to"])

from karateclub import GraphReader
reader = GraphReader("twitch")
target = reader.get_target()

nmi_scores = []
num_runs = 100

for _ in range(num_runs):
    predicted_memberships = [node_cluster_mapping[node] for node in range(len(node_cluster_mapping))]

    nmi = normalized_mutual_info_score(target, predicted_memberships)
    nmi_scores.append(nmi)


In [ ]:
average_nmi = np.mean(nmi_scores)
std_error_nmi = np.std(nmi_scores) / np.sqrt(num_runs)

print("Average NMI:", round(average_nmi,3))
print("Standard Error of NMI:", std_error_nmi)

Average NMI: 0.089
Standard Error of NMI: 1.3877787807814458e-18


## Example: Original use of the 'Karate club' api with dataset load

### Community Detection

In [ ]:
from karateclub import GraphReader

reader = GraphReader("twitch")

graph = reader.get_graph()
target = reader.get_target()

In [ ]:
print(target[:10])

[1 0 1 0 1 0 0 1 1 0]


In [ ]:
from karateclub import SCD

model = SCD()
model.fit(graph)
cluster_membership = model.get_memberships()

In [ ]:
cluster_membership = [cluster_membership[node] for node in range(len(cluster_membership))]
print(cluster_membership)

[3879, 950, 3877, 1681, 3876, 1370, 1743, 3874, 1386, 1514, 3873, 787, 786, 1539, 1565, 1682, 379, 1809, 3867, 29, 1201, 1450, 1449, 407, 687, 3858, 1530, 3854, 3853, 3852, 407, 200, 805, 1511, 883, 1144, 922, 409, 3848, 784, 783, 947, 794, 1526, 782, 1740, 409, 3845, 1619, 199, 3529, 3844, 1123, 1451, 1467, 989, 378, 391, 198, 3843, 3842, 324, 377, 197, 1366, 971, 196, 3841, 369, 1543, 3839, 2206, 2205, 218, 1762, 1094, 195, 194, 3835, 3834, 781, 962, 3879, 1811, 3828, 1396, 987, 3827, 396, 1676, 3826, 800, 3825, 372, 1203, 1675, 1372, 1359, 3801, 946, 990, 3800, 3799, 1924, 409, 433, 1118, 1358, 3797, 3796, 3795, 3794, 1908, 211, 1674, 3793, 191, 3792, 3791, 844, 288, 1673, 1117, 3786, 189, 772, 1356, 1516, 1116, 3784, 1904, 1115, 980, 799, 1094, 771, 3781, 1441, 1711, 121, 3780, 386, 217, 1440, 843, 919, 1026, 770, 1354, 1807, 1353, 3260, 454, 385, 1351, 3854, 3762, 371, 1311, 3134, 768, 939, 1709, 945, 1384, 1806, 211, 1729, 4002, 1339, 3733, 366, 4003, 1531, 948, 3732, 431, 1436, 

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score

# cluster_membership = [cluster_membership[node] for node in range(len(cluster_membership))]

nmi = normalized_mutual_info_score(target, cluster_membership)
print('NMI: {:.3f}'.format(nmi))

NMI: 0.089


### Node Embedding

In [ ]:
from karateclub.dataset import GraphReader

reader = GraphReader("twitch")

graph = reader.get_graph()
y = reader.get_target()

We fit a Diff2vec node embedding, with a low number of dimensions, diffusions per source node, and short Euler walks. First, we use the model constructor with custom parameters. Second, we fit the model to the graph. Third, we get the node embedding which is a numpy array.

In [ ]:
from karateclub import Diff2Vec

model = Diff2Vec(diffusion_number=2, diffusion_cover=20, dimensions=16)
model.fit(graph)
X = model.get_embedding()

We use the node embedding features as predictors of the abusive behaviour. So let us create a train-test split of the explanatory variables and the target variable with Scikit-Learn. We will use a test data ratio of 20%. Here it is.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Using the training data (X_train and y_train) we learn a logistic regression model to predict the probability of someone being an abusive user. We perform inference on the test set for this target. Finally, we evaluate the model performance by printing an area under the ROC curve value.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

downstream_model = LogisticRegression(random_state=0).fit(X_train, y_train)
y_hat = downstream_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_hat)
print('AUC: {:.4f}'.format(auc))

AUC: 0.6129


## Useful Helper functions

In [ ]:
def iterate_nmi(graph, num_runs: int) -> list:
    '''
    Calculates Normalized Mutual Information (NMI) scores for a set of predicted cluster memberships
    against a ground truth, over a specified number of runs.
    Parameters:
    - graph (object): The networkx graph
    - num_runs (int): The number of times to calculate the NMI score. For each run, the function
      uses the `node_cluster_mapping` to simulate predicted memberships and then calculates the
      NMI score against a predefined ground truth.

    Returns:
    - nmi_score (float): The average NMI score over all runs. This value represents the average
      similarity between the predicted cluster assignments in `node_cluster_mapping` and the
      ground truth cluster assignments.

    Example:
    >>> node_cluster_mapping = {0: 1, 1: 2, 2: 1}
    >>> num_runs = 5
    >>> average_nmi = iterate_function(node_cluster_mapping, num_runs)
    >>> print(f"Average NMI score over {num_runs} runs: {average_nmi}")
    '''

    # List of scores
    nmi_scores = []

    # 1. Iterations
    for _ in range(num_runs):

      #  Initialize and run the Label Propagation model
      model = LabelPropagation()
      model.fit(graph)

      # Get prediction
      node_cluster_mapping = model.get_memberships()

      # Take labels or memberships predicted
      predicted_memberships = [node_cluster_mapping[node] for node in range(len(node_cluster_mapping))]
      # Calculate NMI
      nmi = normalized_mutual_info_score(target, predicted_memberships)
      # Append
      nmi_scores.append(nmi)


    # 2. Avg and std
    average_nmi = np.mean(nmi_scores)
    std_error_nmi = np.std(nmi_scores) / np.sqrt(num_runs)

    return (average_nmi, std_error_nmi)
